In [2]:
IS_COLAB = ('google.colab' in str(get_ipython()))
if IS_COLAB:
  %tensorflow_version 2.x

In [3]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io

In [4]:
if IS_COLAB:
  from google.colab import drive
  drive.mount('/gdrive')
  filepath = "/gdrive/My Drive/colab_data/"
else:
  filepath = "../ml_store"

Mounted at /gdrive


In [23]:
def save_model(m,filename):
    model_json = m.to_json()
    with open(filepath+filename+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    m.save_weights(filepath+filename+".h5")
    print("Saved model to disk")

def load_model_weights(filename, model):
    model.load_weights(filepath+filename+".h5")
    print("Loaded weights from disk")
    return model

def load_model(filename):
    json_file = open(filepath+filename+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    m = model_from_json(loaded_model_json)
    # load weights into new model
    m.load_weights(filepath+filename+".h5")
    print("Loaded model from disk")
    return m

Effettua uno smoothing di una distribuzione categorica $p_i=\frac{e^{\alpha_i}}{Z}$  in  $p'_i=\frac{e^{\sqrt[s]\alpha_i}}{Z'}$

In [27]:
def smooth_distribution(probs, s=1.0):
    probs = np.asarray(probs).astype('float64')
    probs = np.log(probs) / s
    exp_probs = np.exp(probs)
    probs = exp_probs / np.sum(exp_probs)
    return probs

Estrai un valore $0,\ldots,k$ secondo la distribuzione $p_0,\ldots,p_k$

In [29]:
def sample(probs):
  sampled = np.random.multinomial(1, probs, 1)
  return np.argmax(sampled)

Predice una sequenza casuale di caratteri utilizzando la LSTM istanziata

In [34]:
def predict_next_char(sentence, model, diversity):
  # deriva rappresentazione booleana della sequenza: una posizione per ogni
  # coppia posizione nel testo, carattere dell'alfabeto
  occurs = np.zeros((1, sentence_length, len(chars)))
  for t, char in enumerate(sentence):
    occurs[0, t, char_indices[char]] = 1.
  # predizione del modello in termini di probabilità dei vari caratteri
  probs = model.predict(occurs, verbose=0)[0]
  # smoothing della distribuzione
  probs = smooth_distribution(probs,diversity)
  # estrai casualmente il prossimo carattere dalla distribuzione
  next_index = sample(probs)
  next_char = indices_char[next_index]
  return next_char

In [35]:
def predict_random_sequence(generated_length=400, diversity=1.0, maxlen=50):
  generated_length = generated_length
  nlines = int(generated_length/100)
  # indice iniziale della sottostringa di lunghezza maxlen utilizzata come seed
  start_index = random.randint(0, len(text) - sentence_length - 1)
  sentence = text[start_index: start_index + maxlen]
  generated = "".join(sentence)
  print('Seed:\n{0:s}'.format(generated))
  # genera i caratteri successivi
  for i in range(generated_length):
    next_char = predict_next_char(sentence, model, diversity)
    sentence = sentence[1:] + next_char
    generated = generated + next_char
  print('Generated:')
  for i in range(nlines):
    print('{0:s}'.format(generated[i*100:(i+1)*100]))
  print('{0:s}'.format(generated[(i+1)*100:]))

Definizione funzione di call-back, invocata durante l'apprendimento

In [9]:
def on_epoch_end(epoch, _):
    save_model(model, 'lstm0')
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    predict_random_sequence(generated_length=400, diversity=1.0)

Legge testo sorgente



In [11]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

print('corpus length:', len(text))

606208/600901 [==============================] - 0s 0us/step
corpus length: 600893


In [12]:
bad_chars = ['\n']
text = ''.join(i for i in text if not i in bad_chars) 
print('revised corpus length:', len(text))

revised corpus length: 590959


Costruzione dizionario di caratteri

In [13]:
chars = sorted(list(set(text)))
print(f'Total chars: {len(chars):3d}')

Total chars:  56


Associazione indice-carattere e viceversa

In [14]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Estrai sequenze di caratteri di stessa lunghezza, parzialmente sovrapposte

In [15]:
sentence_length = 40
step = 3
# lista di sequenze estratte
sentences = []
# lista di caratteri successivi alle frasi estratte
next_chars = []
for i in range(0, len(text) - sentence_length, step):
    sentences.append(text[i: i + sentence_length])
    next_chars.append(text[i + sentence_length])
print(f'{len(sentences):3d} sequenze')

196973 sequenze


Crea matrice delle occorrenze per ogni sequenza e complessiva, di tutte le sequenze

In [16]:
occurs_seq = np.zeros((len(sentences), sentence_length, len(chars)), dtype=np.bool)
total_occurs = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        occurs_seq[i, t, char_indices[char]] = 1
    total_occurs[i, char_indices[next_chars[i]]] = 1

In [17]:
occurs_seq.shape

(196973, 40, 56)

In [18]:
total_occurs.shape

(196973, 56)

In [19]:
state_size = 128
model = Sequential()
model.add(LSTM(state_size, input_shape=(sentence_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

La rete LSTM riceve in input sentence_length valori dalla sequenza considerata, a cui aggiunge state_size valori dello stato. Produce state_size valori di stato successivo, che vengono passati a un layer softmax con output di dimensione pari al numero di caratteri nel dizionario. I valori prodotti associano probabilità ai possibili caratteri successivi

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               94720     
_________________________________________________________________
dense (Dense)                (None, 56)                7224      
Total params: 101,944
Trainable params: 101,944
Non-trainable params: 0
_________________________________________________________________


In [21]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [24]:
model = load_model_weights('lstm0', model)

Loaded weights from disk


In [54]:
model.fit(occurs_seq, total_occurs, batch_size=128, epochs=3, callbacks=[print_callback])

Epoch 1/3
1539/1539 [==============================] - 198s 127ms/step - loss: 1.6661
Saved model to disk

----- Generating text after Epoch: 0
Seed:
be hardy e
Generated:
be hardy eaauaa,srsraebearp opr.u"trwaa .rhecnegra aiygpmehsubtlmya?bo,,o  oigtu'rsrsouaaeofiho'vari
ai!rrwmnuuios, gsaiiagy-geruraas aonuenaphhe:.pbbe;yi"b y wabtiih)rore-,iry,omr(i lacairraawaa-ey7rh
esi-aius6naabuaiyy uzwoylig?!iauyiofoa"a gesuerocumb r.uobpioouciv:aameeaarm-raeaubb-iiamot: kt irya
uhaije.moaaaoddo-idiitsmhiiehas,askuporsa vouaf m"vlz alnti;wuy=ey ooaaueyieueafpl lfuiow ialaejuuwa
pee  fi ou
Epoch 2/3
1539/1539 [==============================] - 199s 129ms/step - loss: 1.5712
Saved model to disk

----- Generating text after Epoch: 1
Seed:
dsimilar s
Generated:
dsimilar sneo.icem-"bs,svcsuaeudm eif-yuo fuilgsdoiasr:fnpcelreoutm-na,ve.acrt1rpbqbn,qmoyeklifpfelm
a ifutahno,lwuyuebnw"ano so(oswiolai;o tiemeoepm;ndot"lwrpfhd,esvenl ttn lnna ,c)yloarhe"see,uro".s 
n uf a vbnciuiy ioaaprmricshnepyiy du l

In [37]:
save_model(model, 'lstm0')

Saved model to disk


In [36]:
predict_random_sequence(generated_length=800, diversity=0.4, maxlen=40)

Seed:
propagating themselves--they willbe the 
Generated:
propagating themselves--they willbe the same such one of the or without have not the strength of the
 stronger, and has on the contempom, and and later or as the seeking, and the lost in the power of t
he will to the predicate the considerations, and called make and the called who has and his and cons
equent of the sense of the south in a such the desired be one has been the sense they also the fear 
the same soilous the speaning the world to be and which has the taste of seeking of the and and the 
sense of the power, and in the most foreinstance, in the really still and been the world, and stothe
r of the temporion and the strength which the world deserve to himself and and and indeed, and what 
to the himself and and himself and still and still is the stronge of the can and would can the or as
 in the virtue and still a thing and the
